# Try Llama Storm for few-shot simplification

## Load Model

I load model through LlamaCpp so it is much lighter

In [64]:
from langchain_community.llms import LlamaCpp

In [65]:
model = LlamaCpp(
            model_path="../models/llama_storm/Llama-3.1-Storm-8B.Q4_K_M.gguf",
            n_gpu_layers=-1,
            n_batch=512,
            temperature=0.8,
            max_tokens=256,
            top_p=5,
            verbose=False,
            n_ctx=8192,
            f16_kv=True,
            repeat_penalty=1.1,
        )

I set the temperature lower because we want the model to follow our instructions strictly.

## Let's try the zero-shot simplification tool by levels

In [82]:
def format_prompt(user_text: str, level: int) -> str:
    if level == 1:
        prompt = (
            """<|start_header_id|>system<|end_header_id|>\n\n"""
            "You are a text simplification tool for the Russian language. "
            "You are given a text in Russian and you need give a simple version of it in Russian. "
            "Your task is to simplify complex sentences into simple ones. "
            "Avoid cramming too many details into one sentence; distribute them across multiple sentences where needed. "
            "Rephrase sentences to remove participial and gerundial constructions. "
            "Where possible, replace passive voice with active voice. "
            "If a sentence consists of only a noun, add a verb. "
            "Replace rare or low-frequency words with more common ones. "
            "Where appropriate, remove or replace foreign words. "
            "Clarify ambiguous phrases by replacing them with more concrete, easily understandable words. "
            "Where possible, avoid words that have paronyms. "
            "Use only Russian, English is forbidden at any cost."
            "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
            "Simplify the following text in Russian: {user_text}\n"
            "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        )
        
    elif level == 2:
        prompt = (
            """<|start_header_id|>system<|end_header_id|>\n\n"""
            "You are a text simplification tool for the Russian language. "
            "You are given a text in Russian and you need give a simple version of it in Russian. "
            "Simplify complex or compound sentences by breaking them into shorter ones, aiming for a sentence length of no more than seven words. "
            "Ensure each sentence contains only one idea. "
            "Avoid participial and gerundial constructions, and prefer active voice over passive voice. "
            "Keep essential information like names, nationalities, and roles. Do not remove important details. "
            "Remove unnecessary foreign words (like brand names) and replace rare or long words with simpler, shorter ones. "
            "Simplify ambiguous phrases by using more concrete, clear words. "
            "Remove minor details that do not add significant meaning, but ensure key information remains intact. "
            "Use only Russian, English is forbidden at any cost."
            "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
            "Simplify the following text in Russian: {user_text}\n"
            "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        )


    elif level == 3:
        prompt = (
            """<|start_header_id|>system<|end_header_id|>\n\n"""
            "You are a text simplification assistant for the Russian language. "
            "You are given a text in Russian and you need give a simple version of it in Russian. "
            "Your task is to make the text as simple as possible. "
            "Each sentence should contain only one idea and be no longer than five words. "
            "Remove or replace foreign words (such as names, places, or brands), and avoid minor details. "
            "Eliminate numbers and remove any unnecessary details. "
            "Focus on using the nominative and genitive cases for nouns, and only the present or past tense for verbs. "
            "Avoid passive voice and inverted word order. "
            "Replace rare or low-frequency words with more common ones. "
            "Where possible, replace complex phrases with common expressions, clichés, or idioms. "
            "Remove any extraneous details (if it is possible to without removing original semantics of sentence) and simplify ambiguous phrases as much as possible."
            "Use only Russian, English is forbidden at any cost."
            "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
            "Simplify the following text in Russian: {user_text}\n"
            "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        )
    
    return prompt.format(user_text=user_text)


### 1 Level

In [83]:
prompt = format_prompt(user_text="Россиянка Елена Максимова одержала победу в международном конкурсе «Миссис Вселенная».", level=1)

In [84]:
model.invoke(str(prompt))

'В международном конкурсе "Миссис Вселенная" победила россиянка Елена Максимова.'

### 2 Level

In [85]:
prompt = format_prompt(user_text="Россиянка Елена Максимова одержала победу в международном конкурсе «Миссис Вселенная».", level=2)

In [86]:
model.invoke(str(prompt))

'Россиянка Елена Максимова стала победительницей конкурса "Миссис Вселенная".'

### 3 Level

In [87]:
prompt = format_prompt(user_text="Россиянка Елена Максимова одержала победу в международном конкурсе «Миссис Вселенная».", level=3)

In [88]:
model.invoke(str(prompt))

'Россиянка одержала победу.'

Well, not bad at all! Let's compute the 200 samples from dataset.

## Load Data


In [97]:
import pandas as pd

data = pd.read_csv("../data/RuSimpleSentAphasia.csv")

data.head()

,source,level 1,level 2,level 3
0,Россиянка Елена Максимова одержала победу в ме...,Россиянка Елена Максимова победила в конкурсе ...,Россиянка победила в конкурсе «Миссис Вселенная».,Россиянка победила в конкурсе «Миссис Вселенная».
1,Представительница России впервые завоевала это...,"В прессе сказали, что участница из России полу...",Участница из России получает этот титул впервые.,Россиянка получает этот титул впервые.
2,"Уточняется, что финал прошел в Софии 4 февраля.",Финал прошел в Болгарии в начале февраля.,Финал был в Болгарии в феврале.,Финал был начале февраля. Он был в в Болгарии.
3,Участие в нем принимали 120 женщин из разных с...,В нем участвовали 120 женщин из разных стран.,В нем участвовали 120 женщин из разных стран.,В нем участвовали женщины из разных стран.
4,«Конкуренция на конкурсе была очень жесткая: р...,«В конкурсе было сложно выиграть. Было много у...,«В конкурсе было сложно выиграть. Было много у...,«В конкурсе было сложно выиграть.


In [98]:
len(data)

1002

In [99]:
new_data = pd.DataFrame(data["source"].head(200))

In [100]:
new_data

,source
0,Россиянка Елена Максимова одержала победу в ме...
1,Представительница России впервые завоевала это...
2,"Уточняется, что финал прошел в Софии 4 февраля."
3,Участие в нем принимали 120 женщин из разных с...
4,«Конкуренция на конкурсе была очень жесткая: р...
...,...
195,Ранее президент Национального института геофиз...
196,Во время операции врачи извлекли 39 металличес...
197,"По словам одного из хирургов, этот пациент поп..."
198,"Выяснилось, что все эти предметы пациент прогл..."


In [102]:
def apply_generation(text, level) -> str:
    prompt = format_prompt(user_text=text, level=level)
    return model.invoke(str(prompt), stop=["\n\n", ". ", " \n\n", ". \n\n", "\n", " \n"])

In [103]:
for level in [1, 2, 3]:
    column = f"level {level}"
    new_data[column] = new_data["source"].apply(lambda x: apply_generation(x, level))

In [104]:
new_data.to_csv("../data/RuSimpleSentAphasia_200_generated.csv", index=False)

## Let's calculate BERTscore between ground truth and predicted texts

In [105]:
new_data

,source,level 1,level 2,level 3
0,Россиянка Елена Максимова одержала победу в ме...,"Россиянка Елена Максимова выиграла конкурс ""Ми...",Елена Максимова из России стала победительнице...,Россиянка Елена Максимова выиграла конкурс.
1,Представительница России впервые завоевала это...,Россиянка впервые получила эту награду,Россиянка-first выиграла эту награду.,Россиянка победила впервые.
2,"Уточняется, что финал прошел в Софии 4 февраля.",Финал состоялся 4 февраля в городе София.,Финал прошел 4 февраля в Софии.,Финал прошел 4 февраля в Софии.
3,Участие в нем принимали 120 женщин из разных с...,В этом мероприятии приняли участие 120 женщин ...,В этом мероприятии приняли участие 120 женщин ...,Женщины из разных стран участвовали в событии.
4,«Конкуренция на конкурсе была очень жесткая: р...,"«Конкурс был очень трудным: много участниц, и ...","«Конкурс был очень сложным: много участниц, но...","«Конкурс был сложным: много участниц, все хоро..."
...,...,...,...,...
195,Ранее президент Национального института геофиз...,Ранее президент итальянского геофизического ин...,В прошлом президент Италии по геофизике и вулк...,Ранее произошло сильное землетрясение
196,Во время операции врачи извлекли 39 металличес...,Во время операции из живота 30-летнего ливанца...,Врачам удалось найти 39 металлических и пластм...,Врачами изъято 39 кусков металла и пластика
197,"По словам одного из хирургов, этот пациент поп...",Пациент был доставлен в больницу с дыхательной...,"Пациент попал в больницу потому, что у него бы...",Пациент попал в больницу с удушьем.
198,"Выяснилось, что все эти предметы пациент прогл...",Пациент проглотил эти предметы за год.,Пациент съел эти предметы за год.,Пациент съел это весь год.


In [ ]:
from evaluate import load
import numpy as np

bertscore = load("bertscore")

bert_scores = {}
val_data = data.head(200)


for level in val_data.columns[1:]:
    references = val_data[level].tolist()
    predictions = new_data[level].tolist()

    results = bertscore.compute(predictions=predictions, references=references, lang="ru")
    
    bert_scores[level] = {
        "precision": np.mean(results['precision']),
        "recall": np.mean(results['recall']),
        "f1": np.mean(results['f1'])
    }

In [118]:
for level, scores in bert_scores.items():
    print(f"BERTScore for {level}:")
    print(f"Precision: {scores['precision']}")
    print(f"Recall: {scores['recall']}")
    print(f"F1: {scores['f1']}\n")

BERTScore for level 1:
Precision: 0.7868844160437584
Recall: 0.792064271569252
F1: 0.7886312890052796

BERTScore for level 2:
Precision: 0.7685994148254395
Recall: 0.7785915061831474
F1: 0.7727830782532692

BERTScore for level 3:
Precision: 0.7649708545207977
Recall: 0.7463380554318428
F1: 0.7548669040203094



The metrics are okay. If we look closely in the data, the generation is logical, the model simplifyes the text. However, we can see some hallucinations (for instance, english inside russian sentences). But due to their small number of appereances,  we can state that they are not the rule but the exception.  
You can examine the dataset yourself in data/RuSimpleSentAphasia_generated.csv